In [1]:
import xarray as xr
import s3fs

In [2]:
s3 = s3fs.S3FileSystem(anon=True)

files = s3.glob("s3://imos-data/IMOS/SRS/SST/ghrsst/L3S-1d/day/2022/")

In [3]:
# init_drop_vars = {'adi_dtime_from_sst', 
#             'aerosol_dynamic_indicator', 
#             'sea_ice_fraction', 
#             'sea_ice_fraction_dtime_from_sst', 
#             'sst_dtime', 'sst_count', 
#             'sst_mean', 
#             'sst_standard_deviation', 
#             'wind_speed', 
#             'wind_speed_dtime_from_sst'}
consistent_vars = {'time',
                    'lat',
                    'lon',
                    'dt_analysis',
                    'l2p_flags',
                    'quality_level',
                    'satellite_zenith_angle',
                    'sea_surface_temperature',
                    'sses_bias',
                    'sses_count',
                    'sses_standard_deviation',
                    'sst_dtime'}

for file in files:
    with s3.open(file) as f:
        print(f"Working on {file}")
        data = xr.open_dataset(f, engine="h5netcdf")
        available_vars = set(data.variables)
        to_drop_vars = list(available_vars - consistent_vars)
        ds = data.drop_vars(to_drop_vars)
        chunked = ds.chunk(chunks={'lat': 1000, 'lon': 1000}) # TODO: optimal chunk?
        for var in chunked.data_vars:
            chunked[var].encoding = {}
        try:
            print("trying appending")
            chunked.to_zarr("s3://imos-data-pixeldrill/alexl", group="sst", mode="a")
        except Exception as e:
            print(f"Failed to write with exception {e}")
# data

Working on imos-data/IMOS/SRS/SST/ghrsst/L3S-1d/day/2022/20220101032000-ABOM-L3S_GHRSST-SSTskin-AVHRR_D-1d_day.nc
trying appending
Working on imos-data/IMOS/SRS/SST/ghrsst/L3S-1d/day/2022/20220102032000-ABOM-L3S_GHRSST-SSTskin-AVHRR_D-1d_day.nc
trying appending
Working on imos-data/IMOS/SRS/SST/ghrsst/L3S-1d/day/2022/20220103032000-ABOM-L3S_GHRSST-SSTskin-AVHRR_D-1d_day.nc
trying appending
Failed to write with exception 'quality_level'
Working on imos-data/IMOS/SRS/SST/ghrsst/L3S-1d/day/2022/20220104032000-ABOM-L3S_GHRSST-SSTskin-AVHRR_D-1d_day.nc
trying appending
Failed to write with exception 'quality_level'
Working on imos-data/IMOS/SRS/SST/ghrsst/L3S-1d/day/2022/20220105032000-ABOM-L3S_GHRSST-SSTskin-AVHRR_D-1d_day.nc
trying appending
Failed to write with exception 'quality_level'
Working on imos-data/IMOS/SRS/SST/ghrsst/L3S-1d/day/2022/20220106032000-ABOM-L3S_GHRSST-SSTskin-AVHRR_D-1d_day.nc
trying appending
Failed to write with exception 'quality_level'
Working on imos-data/IMOS/

FSTimeoutError: 